In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
host = 'P3NWPLSK12SQL-v12.shr.prod.phx3.secureserver.net'
username = 'datastudent'
password = 'dataprog2023!'

In [4]:
import pip
pip.main(["install", "pyodbc"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Collecting pyodbc

Downloading pyodbc-5.1.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata (2.7 kB)

Downloading pyodbc-5.1.0-cp310-cp310-macosx_10_9_x86_64.whl (72 kB)

Output()

Installing collected packages: pyodbc

Successfully installed pyodbc-5.1.0

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip3.10 install --upgrade pip

0

In [7]:
import pyodbc

ImportError: dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyodbc.cpython-310-darwin.so, 2): Library not loaded: /usr/local/opt/unixodbc/lib/libodbc.2.dylib
  Referenced from: /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyodbc.cpython-310-darwin.so
  Reason: image not found

In [3]:
SERVER = 'P3NWPLSK12SQL-v12.shr.prod.phx3.secureserver.net'
DATABASE = 'Northwinds'
USERNAME = 'datastudent'
PASSWORD = 'dataprog2023!'

In [5]:
connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

In [6]:
conn = pyodbc.connect(connectionString)

NameError: name 'pyodbc' is not defined